In [26]:
from pydantic import BaseModel, Field
from typing import List, Optional, Literal
from google import genai
from google.genai import types
from dotenv import load_dotenv
import os

load_dotenv()

True

# SCHEMA Model

In [27]:
class AspectBasedSentimentAnalysisItem(BaseModel):
    aspect: Optional[str] = Field(description="The aspect being analyzed, เฉพาะเจาะจงถึงสิ่งที่ถูกกล่าวถึง เป็นคำนามม ถ้าไม่มีให้ใส่ค่า null.")
    category: Literal["PRODUCT", "SERVICE", "LOCATION", "PRICE"] = Field(description="The category of the aspect.")
    opinion: Optional[str] = Field(default=None, description="The opinion related to the aspect, if any.")
    sentiment: Literal["POSITIVE", "NEGATIVE", "NEUTRAL"] = Field(description="The sentiment associated with the aspect.")

class AspectBasedSentimentAnalysisList(BaseModel):
    elements: List[AspectBasedSentimentAnalysisItem] = Field(description="List of elements for Aspect-Based Sentiment Analysis.")


# GEMINI API

In [28]:
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

In [29]:
SYSTEM_PROMPT = """Your goal is to generate Aspect-based sentiment analysis aims to identify the aspects of given target entities and the sentiment expressed towards each aspect."""

In [30]:
input_text = """ปูนแห้งช้าไปหน่อยคุณภาพดีใช้งานง่าย ช่างมาสาย"""

In [32]:
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=input_text,
    config=types.GenerateContentConfig(
        system_instruction=SYSTEM_PROMPT,
        response_mime_type="application/json",
        response_schema=AspectBasedSentimentAnalysisList,
    ),
)

In [33]:
print(response.text)

{"elements":[{"aspect":"ปูน","category":"PRODUCT","opinion":"แห้งช้าไปหน่อย","sentiment":"NEGATIVE"},{"aspect":"คุณภาพ","category":"PRODUCT","opinion":"ดี","sentiment":"POSITIVE"},{"aspect":"การใช้งาน","category":"PRODUCT","opinion":"ง่าย","sentiment":"POSITIVE"},{"aspect":"ช่าง","category":"SERVICE","opinion":"มาสาย","sentiment":"NEGATIVE"}]}


In [38]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    api_key=os.getenv("GEMINI_API_KEY")
)

In [39]:
llm_strctured_response = llm.with_structured_output(AspectBasedSentimentAnalysisList)

In [41]:
messages = [
    (
        "system",
        SYSTEM_PROMPT,
    ),
    ("human", input_text),
]

In [43]:
response = llm_strctured_response.invoke(messages)

In [45]:
print(response)

elements=[AspectBasedSentimentAnalysisItem(aspect='การแห้งตัว', category='PRODUCT', opinion='แห้งช้าไปหน่อย', sentiment='NEGATIVE'), AspectBasedSentimentAnalysisItem(aspect='คุณภาพ', category='PRODUCT', opinion='ดี', sentiment='POSITIVE'), AspectBasedSentimentAnalysisItem(aspect='การใช้งาน', category='PRODUCT', opinion='ง่าย', sentiment='POSITIVE'), AspectBasedSentimentAnalysisItem(aspect='แรงอัดหลังเซ็ตตัว', category='PRODUCT', opinion='ดีมาก', sentiment='POSITIVE')]
